**Situacion Profesional**

Con lo visto en las clases y el mismo ejemplo que vimos de sistema de recomendacion deben poder crear uno similar
pero mas grande y profesionalizado, considerando que deben mostrarselo a un cliente.
Aconsejo trabajar con a mayor cantidad de datos que puedan, ya que uds los crearan.

Deben suponer que una empresa que brinda el servicio de streaming de películas por internet quiere implementar un Sistema de
Recomendación (SR o RS en inglés) para su usuarios, de tal manera de poder sugerir, dentro de las películas que aún no ha visto, 
cuál podría ser la que más le gustará.
Para ello en su plataforma solicitan a cada usuario que califique la película que acaba de ver.

La calificación (rating) asume valores de 1 a 10, siendo 1 la peor calificación y 10 la mejor.
Se establece el valor 0 para aquellas películas que no han sido calificadas por los usuarios.(debe haber valores nulos)

Ud debe crear una matriz aleatoria de valores que vayan del 1 al 10(puntuacion dada), cuya cantidad de usuarios que votaron
sean mayor a 300 y la cantidad de peliculas votadas sean 20, los nombres de las mismas son indiferentes,
puede ser p1,p2,p3, etc. y d elos usuarios los nombres puede ser u1,u2,u3, etc.

Pueden crear un archivo en excel y luego leerlo en Jupyter, o directamente trabajar sobre una matriz.

Recuerden todas las operaciones realizadas con numpy y pandas.

¡Exitos!

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Abrimos la base de películas vistas (con puntaje del 1 al 10), no vistas (con valores nulos) y no evaluadas (con valor 0)
df_pelis=pd.read_csv("BasePelis00.csv") 

In [3]:
#Identificamos el tamaño de la matriz
df_pelis.shape

(330, 21)

In [4]:
#Visualizamos los primeros registros
df_pelis.head(4)

,Usuario,pel1,pel2,pel3,pel4,pel5,pel6,pel7,pel8,pel9,...,pel11,pel12,pel13,pel14,pel15,pel16,pel17,pel18,pel19,pel20
0,us1,10,NaN,NaN,NaN,0,6,7.0,8.0,10.0,...,4,2,7,4,3,9,6,10,10,10
1,us2,7,5.0,2.0,5.0,3,9,NaN,NaN,NaN,...,2,4,6,4,0,8,1,9,9,7
2,us3,10,3.0,0.0,3.0,1,7,7.0,6.0,0.0,...,5,2,6,3,6,6,10,5,7,8
3,us4,9,2.0,7.0,8.0,9,5,0.0,3.0,5.0,...,6,3,6,10,10,8,8,5,2,6


In [5]:
#Definimos el nombre del Usuario a recomendarle películas de su gusto y lo ponemos en una variable (será el us2)
Usb="us2"
# Aislamos a las observaciones de ese usuario a recomendar en un nuevo data frame para usarlo de base de comparación 
US=df_pelis[df_pelis.Usuario==Usb]
US

,Usuario,pel1,pel2,pel3,pel4,pel5,pel6,pel7,pel8,pel9,...,pel11,pel12,pel13,pel14,pel15,pel16,pel17,pel18,pel19,pel20
1,us2,7,5.0,2.0,5.0,3,9,NaN,NaN,NaN,...,2,4,6,4,0,8,1,9,9,7


In [6]:
#Comenzamos a "limpiar" el dataframe del usuario a recomendar sacando las columnas con valores nulos y guardando sus titulos 
# de columnas en la variable no_vio
no_vio = US.columns[US.isnull().any()]
print(no_vio)
#Eliminamos las columnas con los títulos de películas que no vio para poder depurar y generar una base de comparación 
# para buscar el vector más cercano sin valores nulos
US=US.drop(no_vio, axis=1)
US

Index(['pel7', 'pel8', 'pel9'], dtype='object')


,Usuario,pel1,pel2,pel3,pel4,pel5,pel6,pel10,pel11,pel12,pel13,pel14,pel15,pel16,pel17,pel18,pel19,pel20
1,us2,7,5.0,2.0,5.0,3,9,9,2,4,6,4,0,8,1,9,9,7


In [7]:
#Sacamos las columnas con valor 0 y guardo los titulos en la variable ceros
ceros = US.columns[US.isin([0]).any()]
print(ceros)
# Borramos las columnas con los títulos de las películas que no evaluó para para poder depurar y generar 
# una base de comparación para buscar el vector más cercano sin valores 0 (películas no evaluadas)
US=US.drop(ceros, axis=1)
US

Index(['pel15'], dtype='object')


,Usuario,pel1,pel2,pel3,pel4,pel5,pel6,pel10,pel11,pel12,pel13,pel14,pel16,pel17,pel18,pel19,pel20
1,us2,7,5.0,2.0,5.0,3,9,9,2,4,6,4,8,1,9,9,7


In [8]:
# Generamos otro dataframe sin el usuario a recomendar y sacándole las columnas en que el usuario a recomendar 
# tiene valores 0 o nulos para poder buscar el vector más cercano (usuario con gustos mas parecidos) al usuario a recomendar
UX=UX.drop(ceros, axis=1)
UX=UX.drop(no_vio, axis=1)
UX

,Usuario,pel1,pel2,pel3,pel4,pel5,pel6,pel10,pel11,pel12,pel13,pel14,pel16,pel17,pel18,pel19,pel20
0,us1,10,NaN,NaN,NaN,0,6,8,4,2,7,4,9,6,10,10,10
2,us3,10,3.0,0.0,3.0,1,7,3,5,2,6,3,6,10,5,7,8
3,us4,9,2.0,7.0,8.0,9,5,7,6,3,6,10,8,8,5,2,6
4,us5,9,2.0,2.0,4.0,9,6,1,3,1,7,7,4,7,7,7,1
5,us6,8,0.0,5.0,4.0,0,4,1,0,2,3,3,1,8,4,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,us326,10,10.0,6.0,3.0,0,3,1,3,9,10,3,3,4,7,10,5
326,us327,9,2.0,0.0,6.0,3,4,4,8,10,10,4,10,8,9,6,6
327,us328,9,8.0,4.0,2.0,3,5,7,4,7,6,1,5,0,10,8,1
328,us329,9,5.0,4.0,6.0,10,9,5,6,2,9,10,3,0,7,2,0


In [9]:
#Transformamos los dataframes a array para calcular luego el vector más cercano 
US1=US.to_numpy()
UX1=UX.to_numpy()

In [10]:
#Buscamos el vector mas cercano al usuario a recomendar por aproximaciones sucesivas
minf = 100 
for x in range(0,df_pelis.Usuario.count()-1):
    if np.linalg.norm(US1[0,1:21]-UX1[x,1:21]) < minf:
        minf=np.linalg.norm(US1[0,1:21]-UX1[x,1:21])
        usuar=UX1[x,0:21]
        print("Paso con valores: ", usuar, "y Cercanía vectores: ",minf)
print("Usuario a recomendar:",US1)

Paso con valores:  ['us3' 10 3.0 0.0 3.0 1 7 3 5 2 6 3 6 10 5 7 8] y Cercanía vectores:  13.601470508735444
Paso con valores:  ['us10' 9 9.0 2.0 5.0 5 9 6 7 9 3 3 6 3 1 6 4] y Cercanía vectores:  13.527749258468683
Paso con valores:  ['us11' 9 5.0 3.0 5.0 5 7 5 6 6 3 5 6 3 8 3 4] y Cercanía vectores:  10.63014581273465
Paso con valores:  ['us23' 9 6.0 2.0 7.0 2 8 7 0 6 1 5 6 4 4 8 3] y Cercanía vectores:  10.198039027185569
Paso con valores:  ['us192' 9 5.0 4.0 5.0 6 7 9 7 4 4 4 10 6 10 10 5] y Cercanía vectores:  9.219544457292887
Paso con valores:  ['us200' 7 5.0 2.0 5.0 3 9 9 3 4 6 4 8 1 9 10 7] y Cercanía vectores:  1.4142135623730951
Usuario a recomendar: [['us2' 7 5.0 2.0 5.0 3 9 9 2 4 6 4 8 1 9 9 7]]


In [11]:
#Ponemos en una variable el nombre del usuario más parecido (con gustos parecidos al us2)
UsuarioP=usuar[0]
UsuarioP

'us200'

In [12]:
#Identificamos las películas que no vio el us2 en el más parecido (el 200)
rec=df_pelis.loc[df_pelis["Usuario"]==UsuarioP,no_vio]
rec

,pel7,pel8,pel9
199,10.0,8.0,9.0


In [13]:
#Seleccionamos las péliculas mejor evaluadas (con 9 y 10) por el usuario mas parecido y esas son las que le recomiendo
recn = rec.columns[rec.isin([9,10]).any()]
print("Se le recomienda al us1 la/s siguiente/s película/s :",recn.to_numpy())

Se le recomienda al us1 la/s siguiente/s película/s : ['pel7' 'pel9']
